In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("D://data/churn.csv")

In [ ]:
# We have already performed EDA on this dataset and there is no scope for data cleaning as well

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
round(df.describe(),2).T

In [ ]:
df.head()

In [ ]:
df['Exited'].value_counts()

In [ ]:
# Check the class distribution
sns.countplot(df['Exited'])

In [ ]:
# Baseline accuracy
7963/10000

In [ ]:
df.columns

In [ ]:
dummies = pd.get_dummies(df[['Geography','Gender']], drop_first=True)

In [ ]:
dummies

In [ ]:
x = df.iloc[:,[2,5,6,7,8,9,10,11]]

In [ ]:
x = pd.concat([x,dummies], axis=1)

In [ ]:
y = df['Exited']

In [ ]:
x.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [ ]:
x_train.shape

In [ ]:
y_train.value_counts()

In [ ]:
6346/8000

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()

In [ ]:
model_lr = lr.fit(x_train,y_train)

In [ ]:
model_lr.coef_

In [ ]:
pred_prob=pd.Series(model_lr.predict_proba(x_test)[:,1])

In [ ]:
from sklearn import metrics

In [ ]:
# Look for the fitness of the model before using it for prediction using ROCR/ AUC

# Receiver Operating Characteristic Plot
# Area Under the Curve

ypred = model_lr.predict_proba(x_test)[:,1]
fpr,tpr, _ = metrics.roc_curve(y_test,ypred)
auc = metrics.roc_auc_score(y_test,ypred)
plt.plot(fpr,tpr,label = "Curve, Auc = "+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
pred_churn = model_lr.predict(x_test)

In [ ]:
pred_churn

In [ ]:
pred_churn[:80]

In [ ]:
y_test[:40]

In [ ]:
pd.crosstab(y_test,pred_churn)

In [ ]:
pred_prob.shape

In [ ]:
pred_prob

In [ ]:
pd.crosstab(y_test.reset_index(drop=True),pred_prob>0.5)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
confusion_matrix(y_test,pred_churn)

In [ ]:
(1584+15)/2000

In [ ]:
# Precision 

In [ ]:
15/(15+33)

In [ ]:
from statsmodels import api as sm

In [ ]:
m = sm.GLM(y_train,x_train,family=sm.families.Binomial())

In [ ]:
m1 = m.fit()

In [ ]:
m1.summary()

In [ ]:
m1.fittedvalues()

In [ ]:
pred_churn1 = m1.predict(x_test)

In [ ]:
pd.crosstab(y_test,pred_churn1>0.5)

In [ ]:
accuracy_score(y_test,pred_churn1>0.5)

In [ ]:
metrics.precision_score(y_test,pred_churn1>0.5)

In [ ]:
recall = 55/(55+328)

In [ ]:
# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtree = DecisionTreeClassifier()

In [ ]:
d1 = dtree.fit(x_train,y_train)

In [ ]:
pred_tree = d1.predict(x_test)

In [ ]:
pd.crosstab(y_test,pred_tree)

In [ ]:
(1403+175)/2000

In [ ]:
175/(175+214)

In [ ]:
dtree = DecisionTreeClassifier(max_leaf_nodes=10, max_depth=5, min_samples_leaf=5)

In [ ]:
d2 = dtree.fit(x_train,y_train)

In [ ]:
from dtreeplt import dtreeplt

In [ ]:
dtree_class = dtreeplt(
    model = d2, 
    feature_names=x_train.columns, 
    target_names=np.array(["neg","pos"]))

In [ ]:
dtree_class.view()

In [ ]:
pred_tree1 = d2.predict(x_test)

In [ ]:
pd.crosstab(y_test,pred_tree1)

In [ ]:
(1554+161)/2000

In [ ]:
161/(161+63)

In [ ]:
# Decision Tree pruning
# Grid search / Hyper parameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score

In [ ]:
from sklearn import tree

In [ ]:
params = {'max_depth' : [2,4,6,8,10],'max_leaf_nodes' :[5,10], 'min_samples_leaf' : [2,5]}
clf = tree.DecisionTreeClassifier()
gcv = GridSearchCV(estimator=clf, param_grid=params)
gcv.fit(x_train,y_train)

In [ ]:
d2.get_params().keys()

In [ ]:
gcv.best_estimator_

In [ ]:
model_gcv = gcv.best_estimator_
model_gcv.fit(x_train,y_train)
pred_gcv = model_gcv.predict(x_test)

In [ ]:
accuracy_score(y_test,pred_gcv)

In [ ]:
path = d2.cost_complexity_pruning_path(x_train,y_train)

In [ ]:
alpha = path['ccp_alphas']

In [ ]:
alpha

In [ ]:
acc_train, acc_test = [],[]

for i in alpha:
    tree = DecisionTreeClassifier(ccp_alpha=i)
    tree.fit(x_train,y_train)
    y_train.pred = tree.predict(x_train)
    y_test.pred = tree.predict(x_test)
    
    acc_train.append(accuracy_score(y_train,y_train.pred))
    acc_test.append(accuracy_score(y_test,y_test.pred))
    acc_train.append(metrics.precision_score(y_train,y_train.pred))
    acc_test.append(metrics.precision_score(y_test,y_test.pred))

In [ ]:
acc_train

In [ ]:
alpha

In [ ]:
acc_test

In [ ]:
model_ccp = DecisionTreeClassifier(ccp_alpha=0.00459475)

In [ ]:
model_ccp = model_ccp.fit(x_train,y_train)

In [ ]:
pred_ccp=model_ccp.predict(x_test)

In [ ]:
pd.crosstab(y_test,pred_ccp)

In [ ]:
accuracy_score(y_test,pred_ccp)

In [ ]:
accuracy_score(y_test,pred_tree1)

In [ ]:
accuracy_score(y_test,pred_gcv)

In [ ]:
92/(92+24)

In [ ]:
pd.crosstab(y_test,pred_tree1)

In [ ]:
161/(161+63)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier()

In [ ]:
model_rf = rf.fit(x_train,y_train)

In [ ]:
pred_rf = model_rf.predict(x_test)

In [ ]:
model_rf.feature_importances_

In [ ]:
pd.crosstab(y_test,pred_rf)

In [ ]:
(1555+160)/2000

In [ ]:
160/(160+62)

In [ ]:
df[df['Exited']==1][:6]

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
s = SMOTE()

In [ ]:
x_train_smote, y_train_smote = s.fit_sample(x_train,y_train)

In [ ]:
from collections import Counter

In [ ]:
print("before smote: ", Counter(y_train))
print("after smote: ", Counter(y_train_smote))

In [ ]:
x_train.iloc[:10]

In [ ]:
x_train_smote[:10][:]

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
glm = LogisticRegression()

In [ ]:
model= glm.fit(x_train_smote,y_train_smote)

In [ ]:
pred_glm = model.predict(x_test)

In [ ]:
pd.crosstab(y_test,pred_glm)

In [ ]:
(1072+243)/2000

In [ ]:
243/(243+545)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtree = DecisionTreeClassifier(max_depth=5, max_leaf_nodes=5,min_samples_leaf=5)
dtree=dtree.fit(x_train_smote,y_train_smote)

In [ ]:
pred_smote = dtree.predict(x_test)

In [ ]:
pd.crosstab(y_test,pred_smote)

In [ ]:
199/(199+154)

In [ ]:
from sklearn.svm import SVC

In [ ]:
sv = SVC(kernel='sigmoid')
model_svm = sv.fit(x_train,y_train)

In [ ]:
pred_svm = model_svm.predict(x_test)

In [ ]:
pd.crosstab(y_test,pred_svm)

In [ ]:
# Data Preparation Techniques

# Data Pre-processing : Sampling, Data Transformation --> Standardisation, Normalisation

# Cross-validation Techniques : K-fold, Stratified K-fold, Repeated K-fold

# Feature Selection Techniques : P-value, Step function, K-best, RFE, AUC/ROC

# Handling Class Imbalances : Under Sampling, Over Sampling, SMOTE

# Feature Extraction Techniques : Curse of Dimensionality : PCA, LDA

In [ ]:
# Data Transformation

In [ ]:
# Standardisation : (Center & Scale) : (xi-mean(x))/std(x)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
ss = StandardScaler().fit(x_train)

In [ ]:
ss = ss.transform(x_train)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
model_knn = knn.fit(ss,y_train)

In [ ]:
pred_knn = model_knn.predict(x_test)

In [ ]:
pd.crosstab(y_test,pred_knn)

In [ ]:
# Normalisation --> Range function xi-min(x)/(min(x)-max(x))

In [ ]:
range_x = MinMaxScaler(feature_range=(0,1))
range_x.fit_transform(x_train)

In [ ]:
# Cross Validation Techniques

In [ ]:
# K-fold cv - 3 fold cv

In [ ]:
# 10,000

# 1 fold --> 3500
# 2 fold --> 3250
# 3 fold --> 3250

# 1 model : 1st & 2nd chunk (training) and predicting on the 3rd chunk        --> 85%

# 2nd model : 1st & 3rd chunk (training) and predicting on the 2nd chunk     --> 75%

# 3rd model : 2nd & 3rd (training) and predicting on the 1st chunk            --> 80%

# average of all the models accuarcy

In [ ]:
# Feature Selection Techniques

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, RFE

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()

In [ ]:
rfe_lr = RFE(lr, 5)
result = rfe_lr.fit(x_train,y_train)

In [ ]:
result.n_features_

In [ ]:
result.ranking_

In [ ]:
# Chi2 with Kbest

In [ ]:
model = SelectKBest(score_func=chi2,k=5)
result_kbest = model.fit(x,y)

In [ ]:
result_kbest.scores_

In [ ]:
x_train.columns

In [ ]:
for i in result_kbest.scores_:
    print(round(i,2))

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
model = ExtraTreesClassifier()

In [ ]:
model.fit(x,y)

In [ ]:
model.feature_importances_

In [ ]:
# Evaluation of Algorithms

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

In [ ]:
models = []

In [ ]:
models.append(("LR", LogisticRegression()))
models.append(("Tree", DecisionTreeClassifier()))
models.append(("SVM", SVC()))
models.append(("RF", RandomForestClassifier()))
models.append(("KNN", KNeighborsClassifier()))
models.append(("LDA", LinearDiscriminantAnalysis()))
models.append(("NB", GaussianNB()))
models.append(("XGB", XGBClassifier()))

In [ ]:
models

In [ ]:
results = []
names = []

In [ ]:
from sklearn.model_selection import KFold , cross_val_score, GridSearchCV

In [ ]:
import warnings
warnings.ignore=True

In [ ]:
for name,model in models:
    kfold = KFold(n_splits=5)
    cv_results = cross_val_score(model, x,y,cv=kfold, scoring="accuracy")
    results.append(cv_results)
    names.append(name)
    
    outcome = "%s: %f (%f)" % (name,cv_results.mean()*100, cv_results.std()*100)
    print(outcome)

In [ ]:
cv_results

In [ ]:
help(cross_val_score)

In [ ]:
results

In [ ]:
fig = plt.figure()
axis = fig.add_subplot(111)
plt.boxplot(results)
axis.set_xticklabels(names)
plt.show()

In [87]:
# Explore the pipeline method to evaluate algorithms